In [1]:
import pandas as pd
import numpy as np
import os
os.listdir("G:/Unilever Singapore (15-AIM-1275)/Users/Kumar/Amexpert") #listing input files
#os.listdir("Desktop/Amexpert/")

['Amexpert.ipynb',
 'historical_user_logs.csv',
 'sample_submission_2s8l9nF.csv',
 'test.csv',
 'test_LNMuIYp.zip',
 'train.csv',
 'train_amex.zip']

In [2]:
# Readind files
train_data = pd.read_csv("G:/Unilever Singapore (15-AIM-1275)/Users/Kumar/Amexpert/train.csv")
test_data = pd.read_csv("G:/Unilever Singapore (15-AIM-1275)/Users/Kumar/Amexpert/test.csv")

train_data['data_flag'] = "train data"
test_data['is_click']  = "NA"
test_data['data_flag'] = "test data"


In [3]:
# reading historical Data
hist_data = pd.read_csv("G:/Unilever Singapore (15-AIM-1275)/Users/Kumar/Amexpert/historical_user_logs.csv")
hist_data.head()

,DateTime,user_id,product,action
0,2017-05-28 15:44,704,B,view
1,2017-05-29 07:08,499679,F,view
2,2017-05-29 07:10,499679,G,view
3,2017-05-29 07:10,499679,G,view
4,2017-05-29 07:10,499679,G,view


In [4]:
df = hist_data.iloc[:,1:4].drop_duplicates()
df['flag'] = 1
df = df.pivot_table(index=['user_id','product'],columns='action',values='flag',fill_value=0)
df = df.reset_index()
print(df.columns)


Index(['user_id', 'product', 'interest', 'view'], dtype='object', name='action')


In [5]:
#train_data.head()
#test_data.head()
#train_data.describe()


train_data = train_data.append(test_data)


train_data = pd.merge(train_data,df, how = 'left',on = ['user_id','product'])
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592149 entries, 0 to 592148
Data columns (total 18 columns):
session_id                592149 non-null int64
DateTime                  592149 non-null object
user_id                   592149 non-null int64
product                   592149 non-null object
campaign_id               592149 non-null int64
webpage_id                592149 non-null int64
product_category_1        592149 non-null int64
product_category_2        150124 non-null float64
user_group_id             568222 non-null float64
gender                    568222 non-null object
age_level                 568222 non-null float64
user_depth                568222 non-null float64
city_development_index    432411 non-null float64
var_1                     592149 non-null int64
is_click                  592149 non-null object
data_flag                 592149 non-null object
interest                  478419 non-null float64
view                      478419 non-null float64
dtypes

In [7]:
train_data['view'].fillna(0,inplace= True)
train_data['interest'].fillna(0,inplace= True)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592149 entries, 0 to 592148
Data columns (total 18 columns):
session_id                592149 non-null int64
DateTime                  592149 non-null object
user_id                   592149 non-null int64
product                   592149 non-null object
campaign_id               592149 non-null int64
webpage_id                592149 non-null int64
product_category_1        592149 non-null int64
product_category_2        150124 non-null float64
user_group_id             568222 non-null float64
gender                    568222 non-null object
age_level                 568222 non-null float64
user_depth                568222 non-null float64
city_development_index    432411 non-null float64
var_1                     592149 non-null int64
is_click                  592149 non-null object
data_flag                 592149 non-null object
interest                  592149 non-null float64
view                      592149 non-null float64
dtypes

In [8]:
#imputing missig values for user_group_id

for_imputing = pd.DataFrame(train_data.ix[:,['user_id','user_group_id']].drop_duplicates())
#for_imputing.info()
row = for_imputing.shape[0]
print(row)


for i in range(0,row):
    #print(i)
    if pd.isnull(for_imputing.iloc[i,1]):
        for_imputing.iloc[i,1] = round(np.nansum(for_imputing.iloc[i-1,1]+for_imputing.iloc[i+1,1])/2)


#df1 = train_data.ix[:,['user_group_id','gender','age_level','user_depth']]
#df1.drop_duplicates().sort_values("age_level")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


177729


In [9]:
final_data = pd.merge(train_data,for_imputing, how='left', left_on='user_id', right_on = 'user_id')

final_data['user_group_id'] = final_data['user_group_id_y']
final_data = final_data.drop(['user_group_id_x','user_group_id_y'],axis = 1)

In [10]:
final_data.head()

,session_id,DateTime,user_id,product,campaign_id,webpage_id,product_category_1,product_category_2,gender,age_level,user_depth,city_development_index,var_1,is_click,data_flag,interest,view,user_group_id
0,140690,2017-07-02 00:00,858557,C,359520,13787,4,NaN,Female,4.0,3.0,3.0,0,0,train data,0.0,1.0,10.0
1,333291,2017-07-02 00:00,243253,C,105960,11085,5,NaN,Female,2.0,2.0,NaN,0,0,train data,0.0,1.0,8.0
2,129781,2017-07-02 00:00,243253,C,359520,13787,4,NaN,Female,2.0,2.0,NaN,0,0,train data,0.0,1.0,8.0
3,464848,2017-07-02 00:00,1097446,I,359520,13787,3,NaN,Male,3.0,3.0,2.0,1,0,train data,1.0,1.0,3.0
4,90569,2017-07-02 00:01,663656,C,405490,60305,3,NaN,Male,2.0,3.0,2.0,1,0,train data,0.0,1.0,2.0


In [11]:
# Imputing missing values for gender col

print(pd.crosstab(train_data.user_group_id, train_data.gender))

x = final_data.index[final_data['user_group_id'].isin([1,2,3,4,5,6])]
final_data.loc[x,'gender'] = "Male"
#final_data.loc[x,'gender'].drop_duplicates()

y = final_data.index[final_data['user_group_id'].isin([7,8,9,10,11,12])]
#final_data.loc[y,'gender'].drop_duplicates()
final_data.loc[y,'gender'] = "Female"
#final_data.loc[y,'gender'].drop_duplicates()

# Assumption
# where user Id is 0 and gender is "NA", replcae it with Male
z = final_data.index[final_data['user_group_id'].isin([0]) & pd.isnull(final_data['gender'])]
final_data.loc[z,'gender'] = "Male"

print(pd.crosstab(final_data.user_group_id, final_data.gender))

gender         Female    Male
user_group_id                
0.0                40     164
1.0                 0   53456
2.0                 0  174371
3.0                 0  178102
4.0                 0   64937
5.0                 0   28719
6.0                 0    2230
7.0              1807       0
8.0              7841       0
9.0             25713       0
10.0            17806       0
11.0            11674       0
12.0             1362       0
gender         Female    Male
user_group_id                
0.0                40    1492
1.0                 0   54045
2.0                 0  183654
3.0                 0  182147
4.0                 0   68851
5.0                 0   29288
6.0                 0    5094
7.0              2468       0
8.0              8155       0
9.0             25832       0
10.0            18039       0
11.0            11682       0
12.0             1362       0


In [12]:
print(pd.crosstab(final_data.user_group_id, final_data.age_level))

age_level      0.0    1.0     2.0     3.0    4.0    5.0   6.0
user_group_id                                                
0.0            204      0       0       0      0      0     0
1.0              0  53456       0       0      0      0     0
2.0              0      0  174371       0      0      0     0
3.0              0      0       0  178102      0      0     0
4.0              0      0       0       0  64937      0     0
5.0              0      0       0       0      0  28719     0
6.0              0      0       0       0      0      0  2230
7.0              0   1807       0       0      0      0     0
8.0              0      0    7841       0      0      0     0
9.0              0      0       0   25713      0      0     0
10.0             0      0       0       0  17806      0     0
11.0             0      0       0       0      0  11674     0
12.0             0      0       0       0      0      0  1362


In [13]:
# Imputation Missing for age_level  
def age_level_fg(final_data):

    if (final_data['user_group_id'] == 0):
        return 0
    elif (final_data['user_group_id'] == 1 or  (final_data['user_group_id'] ==7)):
        return 1
    elif (final_data['user_group_id'] == 2 or  (final_data['user_group_id'] ==8)):
        return 2
    elif (final_data['user_group_id'] == 3 or  (final_data['user_group_id'] ==9)):
        return 3
    elif (final_data['user_group_id'] == 4 or  (final_data['user_group_id'] ==10)):
        return 4
    elif (final_data['user_group_id'] == 5 or  (final_data['user_group_id'] ==11)):
        return 5
    elif (final_data['user_group_id'] == 6 or  (final_data['user_group_id'] ==12)):
        return 6
    
final_data['age_level'] = final_data.apply(age_level_fg, axis = 1)    
    
    
    
print(pd.crosstab(final_data.user_group_id, final_data.age_level))

age_level         0      1       2       3      4      5     6
user_group_id                                                 
0.0            1532      0       0       0      0      0     0
1.0               0  54045       0       0      0      0     0
2.0               0      0  183654       0      0      0     0
3.0               0      0       0  182147      0      0     0
4.0               0      0       0       0  68851      0     0
5.0               0      0       0       0      0  29288     0
6.0               0      0       0       0      0      0  5094
7.0               0   2468       0       0      0      0     0
8.0               0      0    8155       0      0      0     0
9.0               0      0       0   25832      0      0     0
10.0              0      0       0       0  18039      0     0
11.0              0      0       0       0      0  11682     0
12.0              0      0       0       0      0      0  1362


In [14]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592149 entries, 0 to 592148
Data columns (total 18 columns):
session_id                592149 non-null int64
DateTime                  592149 non-null object
user_id                   592149 non-null int64
product                   592149 non-null object
campaign_id               592149 non-null int64
webpage_id                592149 non-null int64
product_category_1        592149 non-null int64
product_category_2        150124 non-null float64
gender                    592149 non-null object
age_level                 592149 non-null int64
user_depth                568222 non-null float64
city_development_index    432411 non-null float64
var_1                     592149 non-null int64
is_click                  592149 non-null object
data_flag                 592149 non-null object
interest                  592149 non-null float64
view                      592149 non-null float64
user_group_id             592149 non-null float64
dtypes: 

In [15]:
#imputation missing for user_depth
final_data.loc[final_data.index[pd.isna(final_data['user_depth'])],'user_depth'] = 3

final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592149 entries, 0 to 592148
Data columns (total 18 columns):
session_id                592149 non-null int64
DateTime                  592149 non-null object
user_id                   592149 non-null int64
product                   592149 non-null object
campaign_id               592149 non-null int64
webpage_id                592149 non-null int64
product_category_1        592149 non-null int64
product_category_2        150124 non-null float64
gender                    592149 non-null object
age_level                 592149 non-null int64
user_depth                592149 non-null float64
city_development_index    432411 non-null float64
var_1                     592149 non-null int64
is_click                  592149 non-null object
data_flag                 592149 non-null object
interest                  592149 non-null float64
view                      592149 non-null float64
user_group_id             592149 non-null float64
dtypes: 

In [16]:
pd.crosstab(final_data.city_development_index.fillna('Missing'),final_data.user_depth)

user_depth,1.0,2.0,3.0
city_development_index,,,
1.0,33,794,43166
2.0,152,3308,185819
3.0,162,3168,109515
4.0,288,3187,82819
Missing,14563,28473,116702


In [17]:
pd.crosstab(test_data.city_development_index,test_data.user_group_id)

user_group_id,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
city_development_index,,,,,,,,,,,,,
1.0,5,405,2907,3318,1413,382,18,11,105,347,305,123,10
2.0,4,2098,13987,13487,5340,2123,135,56,431,1657,1337,919,62
3.0,24,2113,7320,7526,2679,1477,97,95,228,1049,917,515,96
4.0,7,1137,4880,6547,2627,950,125,34,343,1273,765,374,66


In [18]:
### This cell predicts the missing value for City_development_index.
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()


data_with_null = final_data[['age_level','user_group_id','user_depth','var_1','city_development_index']]
#data_with_null.shape

data_without_null = data_with_null.dropna()
#data_without_null.shape
train_data_x = data_without_null.iloc[:,:4]
#only city_development_index
train_data_y = data_without_null.iloc[:,4]

#training data
linreg.fit(train_data_x,train_data_y)

#predicting
test_data = data_with_null.iloc[:,:4]
data_with_null['predicted']= pd.DataFrame(linreg.predict(test_data))
data_with_null['predicted'] = round(data_with_null['predicted'])

#data_with_null.city_development_index.fillna(data_with_null.predicted,inplace = True)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
data_with_null

,age_level,user_group_id,user_depth,var_1,city_development_index,predicted
0,4,10.0,3.0,0,3.0,3.0
1,2,8.0,2.0,0,NaN,3.0
2,2,8.0,2.0,0,NaN,3.0
3,3,3.0,3.0,1,2.0,2.0
4,2,2.0,3.0,1,2.0,2.0
5,1,1.0,3.0,0,NaN,3.0
6,3,9.0,3.0,0,4.0,3.0
7,4,4.0,3.0,0,4.0,3.0
8,4,4.0,3.0,0,NaN,3.0
9,4,4.0,3.0,0,4.0,3.0


In [20]:
data_with_null.city_development_index.fillna(data_with_null.predicted,inplace=True)
data_with_null


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,age_level,user_group_id,user_depth,var_1,city_development_index,predicted
0,4,10.0,3.0,0,3.0,3.0
1,2,8.0,2.0,0,3.0,3.0
2,2,8.0,2.0,0,3.0,3.0
3,3,3.0,3.0,1,2.0,2.0
4,2,2.0,3.0,1,2.0,2.0
5,1,1.0,3.0,0,3.0,3.0
6,3,9.0,3.0,0,4.0,3.0
7,4,4.0,3.0,0,4.0,3.0
8,4,4.0,3.0,0,3.0,3.0
9,4,4.0,3.0,0,4.0,3.0


In [21]:
final_data = final_data.drop(['city_development_index'],axis=1)
Final_cleaned_data = pd.concat([final_data.reset_index(drop=True),data_with_null['city_development_index']],axis=1)

In [22]:
Final_cleaned_data.info()
#pd.crosstab(Final_cleaned_data.product_category_2.fillna("Missing"),Final_cleaned_data.webpage_id)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592149 entries, 0 to 592148
Data columns (total 18 columns):
session_id                592149 non-null int64
DateTime                  592149 non-null object
user_id                   592149 non-null int64
product                   592149 non-null object
campaign_id               592149 non-null int64
webpage_id                592149 non-null int64
product_category_1        592149 non-null int64
product_category_2        150124 non-null float64
gender                    592149 non-null object
age_level                 592149 non-null int64
user_depth                592149 non-null float64
var_1                     592149 non-null int64
is_click                  592149 non-null object
data_flag                 592149 non-null object
interest                  592149 non-null float64
view                      592149 non-null float64
user_group_id             592149 non-null float64
city_development_index    592149 non-null float64
dtypes: 

In [23]:
#Splitting entire data into Given Test and Train

Original_Cleaned_Train_data = Final_cleaned_data.loc[Final_cleaned_data['data_flag']== 'train data'].drop(['data_flag','product_category_2','session_id','DateTime'],axis = 1)
Original_Cleaned_Test_data = Final_cleaned_data.loc[Final_cleaned_data['data_flag']== 'test data'].drop(['data_flag','product_category_2','is_click','DateTime'],axis = 1)

print(Original_Cleaned_Train_data.info())
print(Original_Cleaned_Test_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463291 entries, 0 to 463290
Data columns (total 14 columns):
user_id                   463291 non-null int64
product                   463291 non-null object
campaign_id               463291 non-null int64
webpage_id                463291 non-null int64
product_category_1        463291 non-null int64
gender                    463291 non-null object
age_level                 463291 non-null int64
user_depth                463291 non-null float64
var_1                     463291 non-null int64
is_click                  463291 non-null object
interest                  463291 non-null float64
view                      463291 non-null float64
user_group_id             463291 non-null float64
city_development_index    463291 non-null float64
dtypes: float64(5), int64(6), object(3)
memory usage: 53.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 128858 entries, 463291 to 592148
Data columns (total 13 columns):
user_id             

In [25]:
#Original_Cleaned_Train_data splitting into train and test
features =['product', 'campaign_id', 'webpage_id', 'product_category_1',
       'gender', 'age_level','user_depth','var_1', 'user_group_id','interest' ,'view','city_development_index']
Original_Cleaned_Train_data_for_Model = pd.get_dummies(Original_Cleaned_Train_data[features])
Y = np.asarray(Original_Cleaned_Train_data['is_click'])
Y = Y.astype('int')
X = Original_Cleaned_Train_data_for_Model

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)
print("Training and testing split was successful.")

Training and testing split was successful.


In [26]:
# TODO: Import two metrics from sklearn - fbeta_score and accuracy_score
from sklearn.metrics import fbeta_score, accuracy_score,roc_auc_score
def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size' using .fit(training_features[:], training_labels[:])
    start = pd.datetime.now() # Get start time
    learner = learner.fit(X_train[:sample_size],y_train[:sample_size])
    end = pd.datetime.now() # Get end time
    #print(X_train[:sample_size],y_train[:sample_size])
    # TODO: Calculate the training time
    results['train_time'] = end - start
    # TODO: Get the predictions on the test set(X_test),
    #       then get predictions on the first 300 training samples(X_train) using .predict()
    start = pd.datetime.now() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:sample_size])
    end = pd.datetime.now() # Get end time
    # TODO: Calculate the total prediction time
    results['pred_time'] = end - start
            
    # TODO: Compute accuracy on the first 300 training samples which is y_train[:300]
    results['auc_train'] = roc_auc_score(y_train[:sample_size],predictions_train.round())
        
    # TODO: Compute accuracy on test set using accuracy_score()
    results['auc_test'] = roc_auc_score(y_test,predictions_test)
   
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
    print(set(predictions_test))
    print(roc_auc_score(y_test,predictions_test.round()))
    # Return the results
    return results

In [28]:
# Importing the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
#from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# TODO: Initialize the three models
clf_A = DecisionTreeClassifier(random_state=42)
#clf_B = SVC(random_state=42)
clf_C = RandomForestClassifier(n_estimators=100,random_state=42)

#Try -Calculate the number of samples for 1%, 10%, and 100% of the training data
samples_100 = len(X_train)
samples_10 = int(len(X_train)*0.1)
samples_1 = int(len(X_train)*0.01)


# running loop on models
results = {}
for clf in [clf_A, clf_C]:#, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10, samples_100]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)


DecisionTreeClassifier trained on 3706 samples.
{0, 1}
0.49955289031182587
DecisionTreeClassifier trained on 37063 samples.
{0, 1}
0.5004947032822475
DecisionTreeClassifier trained on 370632 samples.
{0, 1}
0.5000469732776355
RandomForestClassifier trained on 3706 samples.
{0, 1}
0.4999976064277928
RandomForestClassifier trained on 37063 samples.
{0, 1}
0.5002933376740716
RandomForestClassifier trained on 370632 samples.
{0, 1}
0.499998735366786


In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

clf = DecisionTreeClassifier(criterion='gini',random_state=42)

parameter_grid = {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8],
                  'max_features': [1, 2, 3, 4,5,6]}
scorer = make_scorer(roc_auc_score)

tree = GridSearchCV(clf, parameter_grid,scorer)

tree.fit(X_train, y_train)
best_clf = tree.best_estimator_
tree_preds = tree.predict(X_test)
tree_performance = roc_auc_score(y_test, tree_preds)

print ("Best Score: {}".format(tree.best_score_))
print ('DecisionTree: Area under the ROC curve = {}'.format(tree_performance))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Best Score: 0.5000153063828604
DecisionTree: Area under the ROC curve = 0.4999826366782808


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

clf = RandomForestClassifier(random_state=42)

n_estimators  = [10,20,30,50,60,70,80]
max_features = ['auto','sqrt']
max_depth = [5,10,15,20,25] # max depth
min_samples_split = [2,4,6] # min samples at each split
min_samples_leaf = [1,2,4,5] # min samples at each node
 
parameter_grid = {'n_estimators' : n_estimators,
               'max_features' : max_features,
               'max_depth' : max_depth,
               'min_samples_split' : min_samples_split,
               'min_samples_leaf' : min_samples_leaf}

scorer = make_scorer(roc_auc_score)

tree = GridSearchCV(clf, parameter_grid,scorer)

tree.fit(X_train, y_train)
best_clf = tree.best_estimator_
tree_preds = tree.predict(X_test)
tree_performance = roc_auc_score(y_test, tree_preds)

print ("Best Score: {}".format(tree.best_score_))
print ('DecisionTree: Area under the ROC curve = {}'.format(tree_performance))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
features =['product', 'campaign_id', 'webpage_id', 'product_category_1',
       'gender', 'age_level','user_depth','var_1', 'user_group_id','interest' ,'view','city_development_index']
session_id = Original_Cleaned_Test_data['session_id']
Original_Cleaned_Test_data = pd.get_dummies(Original_Cleaned_Test_data[features])


In [ ]:
is_click = clf.predict(Original_Cleaned_Test_data)

In [ ]:
Final_submission = pd.concat(session_id,is_click,axis=1)